# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: 
* Model: 
* Evaluation approach: 
* Fine-tuning dataset: 

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [1]:
!pip install -U datasets ## this line is important to resoleve keyError while downloading dataset
!pip install -q "datasets==2.15.0"

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 5.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 464.1/464.1 kB 40.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 13.2 MB/s eta 0:00:00
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.66.2
    Uninstalling tqdm-4.66.2:
      Successfully uninstalled tqdm-4.66.2
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.21.4
    Uninstalling huggingface-hub-0.21.4:
      Successfully uninst

In [2]:
import torch
import numpy as np
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments, DataCollatorWithPadding
from datasets import load_dataset
from peft import LoraConfig, get_peft_model, AutoPeftModelForSequenceClassification


In [3]:
model_name = "bert-base-uncased"
pretrained_model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
tokenizer = AutoTokenizer.from_pretrained(model_name)


/home/student/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [4]:
dataset = load_dataset("imdb")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [6]:
def preprocess_function(examples):
    tokenized = tokenizer(
        examples["text"],
        padding="max_length",
        truncation=True
    )
    if "label" in examples:
        tokenized["labels"] = examples["label"]
    return tokenized

In [7]:
encoded_datasets = dataset.map(preprocess_function, batched=True, remove_columns=["text", "label"])
encoded_datasets.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])


Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [35]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}

In [9]:
training_args = TrainingArguments(
    output_dir="./results",
    per_device_eval_batch_size=8,
)

In [10]:
trainer = Trainer(
    model=pretrained_model,
    args=training_args,
    eval_dataset=encoded_datasets["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [11]:
pretrained_results = trainer.evaluate()
print("Pretrained Model Accuracy:", pretrained_results)

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Pretrained Model Accuracy: {'eval_loss': 0.730438768863678, 'eval_accuracy': 0.49992, 'eval_runtime': 785.7817, 'eval_samples_per_second': 31.815, 'eval_steps_per_second': 3.977}


## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [12]:
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["query", "value"],
    lora_dropout=0.1,
    bias="none",
    task_type="SEQ_CLS"
)


In [13]:
peft_model = get_peft_model(pretrained_model, lora_config)
peft_model.print_trainable_parameters()

trainable params: 297,988 || all params: 109,780,228 || trainable%: 0.27144050019644705


In [14]:
# Fine tuning arguments
training_args = TrainingArguments(
    output_dir="./fine_tuned",
    learning_rate=2e-3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=1,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)


In [15]:
trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=encoded_datasets["train"],
    eval_dataset=encoded_datasets["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

In [16]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.699000,0.693635,0.500000


Checkpoint destination directory ./fine_tuned/checkpoint-3125 already exists and is non-empty.Saving will proceed but saved results may be invalid.


TrainOutput(global_step=3125, training_loss=0.7086805102539062, metrics={'train_runtime': 2571.0576, 'train_samples_per_second': 9.724, 'train_steps_per_second': 1.215, 'total_flos': 6600543744000000.0, 'train_loss': 0.7086805102539062, 'epoch': 1.0})

In [17]:
fine_tuned_results = trainer.evaluate()
print("Model Accuracy:", fine_tuned_results)

Model Accuracy: {'eval_loss': 0.6936354637145996, 'eval_accuracy': 0.5, 'eval_runtime': 786.0568, 'eval_samples_per_second': 31.804, 'eval_steps_per_second': 3.976, 'epoch': 1.0}


In [18]:
peft_model.save_pretrained("./saved_peft_model")
tokenizer.save_pretrained("./saved_peft_model")

print("LoRA model saved!")


LoRA model saved!


## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [19]:
from transformers import AutoTokenizer
import torch
from datasets import load_dataset
from transformers import Trainer, TrainingArguments
import numpy as np

In [21]:
loaded_peft_model = AutoPeftModelForSequenceClassification.from_pretrained("./saved_peft_model")
loaded_peft_model.eval()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): BertForSequenceClassification(
      (bert): BertModel(
        (embeddings): BertEmbeddings(
          (word_embeddings): Embedding(30522, 768, padding_idx=0)
          (position_embeddings): Embedding(512, 768)
          (token_type_embeddings): Embedding(2, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): BertEncoder(
          (layer): ModuleList(
            (0-11): 12 x BertLayer(
              (attention): BertAttention(
                (self): BertSelfAttention(
                  (query): Linear(
                    in_features=768, out_features=768, bias=True
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=768, out_

In [36]:
def tokenize_fn(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=512)

In [37]:
tokenized_test_dataset = dataset["test"].map(tokenize_fn, batched=True)

In [38]:
tokenized_test_dataset.set_format("torch")

In [39]:
trainer = Trainer(
    model=loaded_peft_model,
    eval_dataset=tokenized_test_dataset,
    compute_metrics=compute_metrics
)

In [40]:
evaluation_results = trainer.evaluate()
print("Fine-Tuned Model Evaluation Results:", evaluation_results)

Fine-Tuned Model Evaluation Results: {'eval_loss': 0.6936354637145996, 'eval_accuracy': 0.5, 'eval_runtime': 782.2753, 'eval_samples_per_second': 31.958, 'eval_steps_per_second': 3.995}


In [53]:
pretrained_model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [54]:
dataset = load_dataset("imdb", split="test[:10%]")

In [55]:
trainer_pretrained = Trainer(
    model=pretrained_model,
    eval_dataset=tokenized_test_dataset,
    compute_metrics=compute_metrics
)

In [56]:
pretrained_results = trainer_pretrained.evaluate()
print("Pretrained Model Evaluation Results:", pretrained_results)

Pretrained Model Evaluation Results: {'eval_loss': 0.7204952836036682, 'eval_accuracy': 0.49992, 'eval_runtime': 788.1549, 'eval_samples_per_second': 31.72, 'eval_steps_per_second': 3.965}


In [57]:
print("\n*** Model Comparison ***")
print(f"Pretrained Model Accuracy: {pretrained_results['eval_accuracy']:.4f}")
print(f"Fine-Tuned Model Accuracy: {evaluation_results['eval_accuracy']:.4f}")


*** Model Comparison ***
Pretrained Model Accuracy: 0.4999
Fine-Tuned LoRA Model Accuracy: 0.5000


In [59]:
loaded_peft_model = loaded_peft_model.to("cpu")

# Tokenizing the input and keep it on the CPU
input_text = "Rick and Morty are crazy"
inputs = tokenizer(input_text, return_tensors="pt").to("cpu")  # Keep inputs on CPU

#  Performing inference to loaded/saved peft model
outputs = loaded_peft_model(**inputs)
logits = outputs.logits
probs = torch.nn.functional.softmax(logits, dim=-1)
predicted_class = torch.argmax(probs, dim=-1).item()

print(f"Predicted class: {predicted_class}")

Predicted class: 0


In [ ]:
# Compltd with Peft model, inference and saving trained model in local root directory